# Coverage Monitor

Problem statement

Approach

Who are we?

## Constants

In [145]:
import os

# benchmark directory
benchmark_loc = "/Users/IceCream/Documents/hw22_fuzzing/benchmark"

# studied project
proj = "isort_setting3"
proj_loc = os.path.join(benchmark_loc, proj)

# output directory
out_loc = os.path.join(proj_loc, "analysis")
if not os.path.exists(out_loc):
    os.makedirs(out_loc)

## Lauching AFL

In [ ]:
# afl driver goes here

## Monitoring executions

In [ ]:
%run cov_tracker.ipynb

# set global cov dict
process_cov(proj_loc, out_loc)

# df_ = pd.DataFrame()
# df_.to_csv(file_name)

# output two files, coverage.csv and exec_count.csv